
# Importing Related Libraries

In [11]:
# Import dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psutil
import time
process = psutil.Process()
!
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore")

from pandas import set_option
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics, tree
from sklearn.tree import DecisionTreeClassifier, export_graphviz 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import precision_recall_curve, f1_score, auc, roc_curve, roc_auc_score

# Exploratory Data Analysis (EDA)

In [15]:
import pandas as pd
diabetes_data = pd.read_csv('C:/Users/vaibh/Desktop/NEHA_PROJECT/diabetes.csv') # Import diabetes dataset
diabetes_data.head(10)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/vaibh/Desktop/NEHA_PROJECT/diabetes.csv'

In [ ]:
# Information of datasets
print("The scale of data is", diabetes_data.shape, "\n")
diabetes_data.info()

In [ ]:
# Statistical of datasets
# Drop the PatientID columns and transpose the data
diabetes_data.drop(columns=['PatientID']).describe().T

In [ ]:
# Reviewing Skew of Attribute Distribution
skew_value = diabetes_data.skew()

In [ ]:
skewness_values = pd.DataFrame({
    'Variable': diabetes_data.columns,
    'Skewness': skew_value
})
plt.figure(figsize=(20,8))
# Plot the skewness values using a barplot
sns.barplot(x='Variable', y='Skewness', data=skewness_values)

# Add labels and a title to the plot

plt.xlabel("Features",size=15)
plt.ylabel("Skewness",size=15)
plt.title("Skewness of Variables in Diabetes Dataset",size=20)

plt.show()

# Data Visualization

In [ ]:
# Histrogram for each attribute in dataset
diabetes_data.hist(figsize=(15,15),column=diabetes_data.drop(columns=['PatientID']).columns,color='purple')
plt.show()

In [ ]:
# Class distribution to preview diabetic statistics
result = diabetes_data.groupby('Diabetic').size()
print(result)
ax = result.plot(kind='bar',color='purple',label='0 - No Diabetic')
ax = result.plot(kind='bar',color='purple',label='1 - Diabetic')
ax.legend(loc='best')



In [ ]:
# Define the feature matrix and target vector before feature selection
X = diabetes_data.drop(columns=['Diabetic'])
y = diabetes_data['Diabetic']
print(X,y)

In [ ]:
# Box and Whisker Plots
X.plot(figsize=(20,20),kind='box',subplots=True,layout=(3,3),sharex=False)
plt.show()

# Feature Selection Method 

### Filter Method
Filter methods are a type of feature selection method that works by selecting features based on some criteria prior to building the model. 

In our example dataset, all of the features are numeric.
Hence, we cannot remove any feature from feature matrix through filter method

### Pearson’s correlation

Correlation between each features

In [ ]:
# Reviewing Correlation between attributes
correlations = round(X.corr(),4)
correlations # 1 represent full positive correlation, -1 represent negative correllation


In [ ]:
# Visualize the resulting correlation matrix using a heatmap:

plt.figure(figsize=(15,12))
sns.heatmap(correlations, annot=True,cmap ='RdBu_r')
plt.show()

In [ ]:
# Loop over bottom diagonal of correlation matrix
for i in range(len(correlations.columns)):
    for j in range(i):
        # Print variables with high correlation
        if abs(correlations.iloc[i, j]) > 0.7:
            print(correlations.columns[i], correlations.columns[j], correlations.iloc[i, j])
else:
    print('The coefficient of each features not greater than 0.7')

Normally, we define high correlation as having a coefficient of greater than 0.7 or less than -0.7. We use loop to check each feature correlation matrix and realize unneccessary to drop any feature since the each features correlation not greater than 0.7.

Correlation between features and target

In [ ]:
# Merge features and targets together for checking correlation
X_y = X.copy()
X_y['Diabetic'] = y
print(X_y)

In [ ]:
correlation = X_y.corr()
correlation_target = correlation[['Diabetic']].drop(labels=['Diabetic'])
correlation_target

In [ ]:
sns.heatmap(correlation_target, annot=True, fmt='.3', cmap='RdBu_r')
plt.show()

From the heatmap above, we can observe the correlation coefficient of PatientID is smaller. We can consider to remove it from datasets if we want to save the times and costs in model training. 

In [ ]:
X = X.drop(columns=['PatientID'])
print(X.columns) # Check the feature matrix
X.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaler = pd.DataFrame(scaler.fit_transform(X),columns=['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure',
       'TricepsThickness', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age'])
X_scaler.head()

# Importing model training and feature selection libraries

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score
# from sklearn.model_selection import cross_val_score
# from mlxtend.feature_selection import SequentialFeatureSelector as SFS
# from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
# from sklearn.model_selection import GridSearchCV

### Wrapper Method Function (Sequential Forward Selection with Supervised Learning Algorithms)

In [ ]:
def feature_selection(dt_model, x_data):
    
    # Ensure the random distributions are same when execute 
    np.random.seed(0)
    
    sfs = SFS(dt_model,
          k_features=8,
          forward=True,
          floating=False,
          scoring='accuracy',
          cv=5)
    
    sfs.fit(x_data, y)
    
    print('Feature selection: \n\n')
    for idx, score in sfs.subsets_.items():
    	print(idx,':', score, '\n')
    
    print('Model Accuracy: \n\n')
    for idx, score in sfs.subsets_.items():
        print(idx,':', score['avg_score'].round(4)*100, '%')
    
    plot_sfs(sfs.get_metric_dict())
    plt.grid()
    plt.title('Accuracy vs Number of Features')
    plt.show()

### Define the confusion matrix plotting Function

In [ ]:
def conf_matrix_plot(conf_matrix):
    
    ax = sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues')

    ax.set_title('Confusion Matrix with labels\n\n');
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');

    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels(['Negative','Positive'])
    ax.yaxis.set_ticklabels(['Negative','Positive'])

    ## Display the visualization of the Confusion Matrix.
    plt.plot(conf_matrix)
    plt.show()

In [ ]:
def class_report_plot(class_report):
    
    plt.figure(figsize=(10,3))
    plt.title('Model Classification Report\n\n')
    sns.heatmap(class_report, annot=True,  fmt='g')
    plt.show()

### Define the AUC ROC plotting function

In [ ]:
def auc_roc(model,y_test,y_predicted):
    
    fpr, tpr, _ = metrics.roc_curve(y_test, y_predicted)
    auc = round(metrics.roc_auc_score(y_test, y_predicted), 4)
    plt.plot(fpr,tpr,color='red', linestyle='--', marker='o', linewidth=2, label= model+" AUC="+str(auc))
    plt.grid()
    plt.legend()
    plt.title('AUC ROC of '+ model)
    plt.xlabel('False Positive')
    plt.ylabel('True Positive')
    plt.show()

### Decision Tree Algorithm


In [ ]:
# Decision Tree Algorithm
start_time = time.time()
initial_memory = process.memory_info().rss

dt_model = DecisionTreeClassifier()
    
# Import feature_selection function
feature_selection(dt_model, X)

In [ ]:
 # From wrapper method, we know which features removed was useful to increase accuracy of model.
X_data = X[['Pregnancies', 'PlasmaGlucose', 'TricepsThickness', 'SerumInsulin', 'BMI', 'Age']]

# Split 80% data for training set
X_rem, X_test, y_rem, y_test = train_test_split(X_data, y, test_size=0.25, random_state=6)

# Split 10% data for validation set and 10% for test set
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=0.9, random_state=6)

dt_model.fit(X_train,y_train)

# Using validation set to observe the accuracy during model training
y_val_pred = dt_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after feature selection: ", round(accuracy*100,2), "%\n")

In [ ]:
# np.random.seed(0)

# parameters = {
#     'max_depth': [2,4,6,8,10],
#     'max_leaf_nodes': [10,15,20,25,30,35,40]
# }

# dt_model = DecisionTreeClassifier()
# random = RandomizedSearchCV(dt_model, parameters, n_iter=30)
# random.fit(X_train,y_train)

# df = pd.concat([pd.DataFrame(random.cv_results_['params']), pd.DataFrame(random.cv_results_['mean_test_score']*100, columns=['Accuracy'])] ,axis=1)
# display(df.sort_values('Accuracy', ascending = False))

In [ ]:
dt_model = DecisionTreeClassifier(max_depth=10, max_leaf_nodes=40)

dt_model.fit(X_train,y_train)

# Using validation set to get the accuracy after model training
y_val_pred = dt_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after hyperparameter tuning: ", round(accuracy*100,2), "%\n")

y_predicted_dt = dt_model.predict(X_test)
y_prob_dt = dt_model.predict_proba(X_test)[:, 1]  

accuracy = accuracy_score(y_test, y_predicted_dt)
print("Decision Tree Model Test Set Accuracy: ", round(accuracy*100,2), "%\n")

conf_matrix = confusion_matrix(y_test, y_predicted_dt)
print("Decision Tree Model Confusion Matrix: \n")
conf_matrix_plot(conf_matrix)

class_report = pd.DataFrame(classification_report(y_test, y_predicted_dt, output_dict=True)).transpose()
print("Decision Tree Model Classification Report: \n")
class_report_plot(class_report)
    
end_time = time.time()
run_time = end_time - start_time

final_memory = process.memory_info().rss
memory_usage = final_memory - initial_memory

print("Memory usage:", memory_usage, 'bytes\n')
print("Runtime:", round(run_time,2), 's')

### K Nearest Neighbors Algorithm


In [ ]:
start_time = time.time()
initial_memory = process.memory_info().rss

knn_model = KNeighborsClassifier()

feature_selection(knn_model, X_scaler)

NameError: name 'feature_selection' is not defined

In [ ]:
X_data = X_scaler[['Pregnancies', 'SerumInsulin', 'BMI', 'Age']]

# Split 80% data for training set
X_rem, X_test, y_rem, y_test = train_test_split(X_data, y, test_size=0.25, random_state=6)

# Split 10% data for validation set and 10% for test set
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=0.9, random_state=6)

knn_model = KNeighborsClassifier()
knn_model.fit(X_train,y_train)

# Using validation set to observe the accuracy during model training
y_val_pred = knn_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after feature selection: ", round(accuracy*100,2), "%\n")

In [ ]:
# Define the best value of K in KNN
score_list = []
for k in range(1,50):
    model = KNeighborsClassifier(n_neighbors=k)
    model.fit(X_train,y_train)
    score = model.score(X_val, y_val)
    score_list.append(score*100)

max_score = max(score_list)

k = [idx+1 for idx, score in enumerate(score_list) if score == max_score]
print(f'The maximum accuracy is {max_score} when the K value is {k} after feature selection.\n')

plt.figure(figsize=(12,8))
plt.plot(range(1,50), score_list, color='blue', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.title('Accurracy vs K value',size=20)
plt.xlabel('K',size=18)
plt.ylabel('Accuracy Percentage',size=18)
plt.grid()
plt.show()

In [ ]:
knn_model = KNeighborsClassifier(6)

knn_model.fit(X_train,y_train)

# Using validation set to get the accuracy after model training
y_val_pred = knn_model.predict(X_val)
y_prob_knn = knn_model.predict_proba(X_test)[:, 1]  
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after hyperparameter tuning: ", round(accuracy*100,2), "%\n")

y_predicted_knn = knn_model.predict(X_test)
accuracy = accuracy_score(y_test, y_predicted_knn)
print("K Nearest Neighbor Model Test Set Accuracy: ", round(accuracy*100,2), "%\n")

conf_matrix = confusion_matrix(y_test, y_predicted_knn)
print("K Nearest Neighbor Model Confusion Matrix: \n")
conf_matrix_plot(conf_matrix)

class_report = pd.DataFrame(classification_report(y_test, y_predicted_knn, output_dict=True)).transpose()
print("K Nearest Neighbor Model Classification Report: \n")
class_report_plot(class_report)


end_time = time.time()
run_time = end_time - start_time

final_memory = process.memory_info().rss
memory_usage = final_memory - initial_memory

print("Memory usage:", memory_usage, 'bytes\n')

print("Runtime:", round(run_time,2), 's')

### Logistic Regression Algorithm

In [ ]:
# Logistic Regression Algorithm
start_time = time.time()
initial_memory = process.memory_info().rss

# The initial value of maximum iteration cannot converge
lr_model = LogisticRegression(max_iter=1000)

# Import feature_selection function
feature_selection(lr_model, X)

In [ ]:
# Split 80% data for training set
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.25, random_state=6)

# Split 10% data for validation set and 10% for test set
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=0.9, random_state=6)

lr_model.fit(X_train,y_train)

# Using validation set to observe the accuracy during model training
y_val_pred = lr_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after feature selection: ", round(accuracy*100,2), "%\n")

In [ ]:
train_array = []
test_array = []
C_array = [0.001, 0.01, 0.1, 1, 10, 100]

for x in C_array:
    clf_new = LogisticRegression(max_iter=1000, C=x)
    clf_new.fit(X_train,y_train)
    y_pred_train = clf_new.predict(X_train)
    train_array.append(accuracy_score(y_pred_train,y_train))
    y_pred_val = clf_new.predict(X_val)
    test_array.append(accuracy_score(y_pred_val,y_val))

plt.plot(C_array,train_array, color='blue', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.plot(C_array,test_array, color='orange', linestyle='dashed', marker='o',markerfacecolor='red', markersize=10)
plt.legend(['Training set', 'Validation set'])
plt.title('Choose the range of best C value')
plt.xlabel('Range of C in log format')
plt.ylabel('Accuracy')
plt.grid()
plt.xscale('log')
plt.show()

In [ ]:
# C_range = np.logspace(-2,2,100)

# parameters = {'C': C_range, 'penalty':['l1','l2'], 'solver':['saga','lbfgs']}

# lr_model = LogisticRegression(max_iter=1000)

# grid = GridSearchCV(lr_model, parameters, cv=3)

# grid.fit(X_train,y_train)

# print("Best parameters:", grid.best_params_, "\n")
# print("Best score:", round(grid.best_score_*100,2), "%\n")

In [ ]:
# Logistic Regression Algorithm
lr_model = LogisticRegression(max_iter=10000,C=0.059,penalty='l2',solver='lbfgs')

lr_model.fit(X_train,y_train)

# Using validation set to get the accuracy after model training
y_val_pred = lr_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after hyperparameter tuning: ", round(accuracy*100,2), "%\n")

# Using test set to get the accuracy after model training

y_predicted_lr = lr_model.predict(X_test)
y_prob_lr = lr_model.predict_proba(X_test)[:, 1]  
accuracy = accuracy_score(y_test,y_predicted_lr)
print("Logistic Regression Model Test set accuracy: ", round(accuracy*100,2), "%\n")

conf_matrix = confusion_matrix(y_test, y_predicted_lr)
print("Logistic Regression Model Confusion Matrix: \n")
conf_matrix_plot(conf_matrix)

class_report = pd.DataFrame(classification_report(y_test,y_predicted_lr, output_dict=True)).transpose()
print("Logistic Regression Model Classification Report: \n")
class_report_plot(class_report)


end_time = time.time()
run_time = end_time - start_time

final_memory = process.memory_info().rss
memory_usage = final_memory - initial_memory

print("Memory usage:", memory_usage, 'bytes\n')

print("Runtime:", round(run_time,2), 's')

### Random Forests Algorithm

In [ ]:
start_time = time.time()
initial_memory = process.memory_info().rss

rf_model = RandomForestClassifier()

feature_selection(rf_model,X)

In [ ]:
X_data = X[['Pregnancies', 'PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI', 'Age']]
   
# Split 80% data for training set
X_rem, X_test, y_rem, y_test = train_test_split(X_data, y, test_size=0.25, random_state=10)

# Split 10% data for validation set and 10% for test set
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=0.9, random_state=10)

rf_model.fit(X_train,y_train)

# Using validation set to observe the accuracy during model training
y_val_pred = rf_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after feature selection: ", round(accuracy*100,2), "%\n")

In [ ]:
accuracy_train = []
accuracy_test = []

np.random.seed(0)
depths = range(1,40)
for i in depths:
    crf = RandomForestClassifier(max_depth=i)
    crf.fit(X_train, y_train)
    accuracy_test.append(accuracy_score(y_val, crf.predict(X_val)))
    accuracy_train.append(accuracy_score(y_train, crf.predict(X_train)))

max_accuracy = np.max(accuracy_test)*100
max_depth = np.argmax(accuracy_test)

print('Validation set accuracy is', round(max_accuracy,3) ,'% after the maximum depth of the tree was defined is', max_depth)

plt.figure(figsize=(10,10))
plt.plot(depths,accuracy_train,'bo-',depths,accuracy_test,'r*:')
plt.grid()
plt.legend(['Training set','Validation set'])
plt.title('Accuracy vs Depth of the tree')
plt.xlabel('Depths')
plt.ylabel('Accuracy')

plt.show()

In [ ]:
# np.random.seed(0)
# parameters = {
#         'n_estimators': (100,300,500,700,900,1000),
#         'min_samples_split': (1,3,5,7,9,),
#         'min_samples_leaf':(1,3,5,7,9),
#         'max_features':(0.5,0.8,'auto'),
#         'max_depth':('None',25,30,35,40)
# }

# grid = GridSearchCV(rf_model, parameters, cv=2)

# grid.fit(X_train,y_train)

# print("Best parameters:", grid.best_params_)
# print("Best score of training data:", round(grid.best_score_*100,2), "%\n")


In [ ]:
rf_model = RandomForestClassifier(
                                  max_depth=40,
                                  max_features='auto',
                                  n_estimators=100,
                                  min_samples_leaf=1,
                                  min_samples_split=1,
                                 )

rf_model.fit(X_train,y_train)

y_val_pred = rf_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after hyperparameter tuning: ", round(accuracy*100,2), "%\n")

y_predicted_rf = rf_model.predict(X_test)
y_prob_rf = rf_model.predict_proba(X_test)[:, 1] 
accuracy = rf_model.score(X_test,y_test)*100
print("Random Forest Model Test Set Accuracy:", round(accuracy,2), "%\n")

conf_matrix = confusion_matrix(y_test, y_predicted_rf)
print("Random Forest Model Confusion Matrix: \n")
conf_matrix_plot(conf_matrix)

class_report = pd.DataFrame(classification_report(y_test, y_predicted_rf, output_dict=True)).transpose()
print("Random Forest Model Classification Report: \n")
class_report_plot(class_report)
    


end_time = time.time()
run_time = end_time - start_time

final_memory = process.memory_info().rss
memory_usage = final_memory - initial_memory

print("Memory usage:", memory_usage, 'bytes\n')

print("Runtime:", round(run_time,2), 's')

### Support Vector Machine (SVM) Algorithm

In [ ]:
start_time = time.time()
initial_memory = process.memory_info().rss

# Initialize the value of C, kernel & gamma
svm_model = SVC(C=1,kernel='rbf',gamma=0.001)

feature_selection(svm_model, X)

In [ ]:
X_data = X[['Pregnancies', 'PlasmaGlucose', 'SerumInsulin', 'BMI', 'DiabetesPedigree', 'Age']]
           
# Split 25% data for test set
X_rem, X_test, y_rem, y_test = train_test_split(X_data, y, test_size=0.25, random_state=6)

# Split 10% data for validation set from training set
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=0.9, random_state=6)

svm_model.fit(X_train,y_train)
           
# Using validation set to observe the accuracy during model training
y_val_pred = svm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy before feature selection: ", round(accuracy*100,2), "%\n")

In [ ]:
# parameters = {'C':[1,10,100,1000],
#                   'kernel':['linear','rbf'],
#                   'gamma':[0.001,0.0001]}

# grid = GridSearchCV(svm_model, parameters, cv=2)

# grid.fit(X_train,y_train)

# print("Best parameters:", grid.best_params_)
# print("Best score of training data:", round(grid.best_score_*100,2), "%\n")

In [ ]:
svm_model = SVC(C=1000,
               kernel='rbf',
               gamma=0.0001,
               probability=True)

svm_model.fit(X_train,y_train)


y_predicted_svm = svm_model.predict(X_test)
y_prob_svm = svm_model.predict_proba(X_test)[:, 1]  
accuracy = svm_model.score(X_val,y_val)*100
print("Support Vector Machine Model Validation Set Accuracy:", round(accuracy,2), "%\n")

#  the accuracy of the final model on the test data
accuracy = svm_model.score(X_test,y_test)*100
print("Support Vector Machine Model Test Set Accuracy:", round(accuracy,2), "%\n")

conf_matrix = confusion_matrix(y_test, y_predicted_svm)
print("Support Vector Machine Model Confusion Matrix: \n")
conf_matrix_plot(conf_matrix)

class_report = pd.DataFrame(classification_report(y_test, y_predicted_svm, output_dict=True)).transpose()
print("Support Vector Machine Model Classification Report: \n")
class_report_plot(class_report)
    


end_time = time.time()
run_time = end_time - start_time

final_memory = process.memory_info().rss
memory_usage = final_memory - initial_memory

print("Memory usage:", memory_usage, 'bytes\n')

print("Runtime:", round(run_time,2), 's')

### XGBoost Algorithm

In [ ]:
start_time = time.time()
initial_memory = process.memory_info().rss

xgb_model = XGBClassifier()

feature_selection(xgb_model,X)

In [ ]:
# Split 25% data for test set
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.25, random_state=6)

# Split 10% data for validation set from training set
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=0.9, random_state=6)

xgb_model.fit(X_train,y_train)
           
# Using validation set to observe the accuracy during model training
y_val_pred = xgb_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after feature selection: ", round(accuracy*100,2), "%\n")

In [ ]:
# np.random.seed(0)
# parameters = {
#         'colsample_bytree': (0.1,0.2,0.3,0.4,0.5),
#         'max_depth': [2,4,6,8,10],
#         'min_child_weight': [1,3,5,7,9],
#         'subsample':(0.5,0.6,0.7,0.8),
#         'gamma':(0,0.1,0.2,0.3,0.5),
#         'learning_rate':(0.1,0.2,0.3)  
# }

# random = RandomizedSearchCV(xgb_model, parameters, n_iter=30, cv=5)

# random.fit(X_train,y_train)

# df = pd.concat([pd.DataFrame(random.cv_results_['params']), pd.DataFrame(random.cv_results_['mean_test_score']*100, columns=['Accuracy'])] ,axis=1)
# display(df.sort_values('Accuracy', ascending = False))

In [ ]:
xgb_model = XGBClassifier( 
                         max_depth=4, 
                         subsample=0.7,
                         min_child_weight=1,
                         learning_rate=0.2,
                         gamma=0.2,
                         colsample_bytree=0.2
                         )random

xgb_model.fit(X_train,y_train)

y_val_pred = xgb_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after hyperparameter tuning: ", round(accuracy*100,2), "%\n")

y_predicted_xgboost = xgb_model.predict(X_test)
y_prob_xgboost = xgb_model.predict_proba(X_test)[:, 1]  
accuracy = xgb_model.score(X_test,y_test)*100
print("XGBoost Model Test set accuracy: ", round(accuracy,2), "%\n")

conf_matrix = confusion_matrix(y_test, y_predicted_xgboost)
print("XGBoost Model Confusion Matrix: \n")
conf_matrix_plot(conf_matrix)

class_report = pd.DataFrame(classification_report(y_test, y_predicted_xgboost, output_dict=True)).transpose()
print("XGBoost Model Classification Report: \n")
class_report_plot(class_report)

end_time = time.time()
run_time = end_time - start_time

final_memory = process.memory_info().rss
memory_usage = final_memory - initial_memory

print("Memory usage:", memory_usage, 'bytes\n')

print("Runtime:", round(run_time,2), 's')

### LGBM Algorithm

In [ ]:
start_time = time.time()
initial_memory = process.memory_info().rss

lgbm_model = LGBMClassifier()

feature_selection(lgbm_model,X)

In [ ]:
# Split 25% data for test set
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=0.25, random_state=6)

# Split 10% data for validation set from training set
X_train, X_val, y_train, y_val = train_test_split(X_rem, y_rem, train_size=0.9, random_state=6)

lgbm_model.fit(X_train,y_train)
           
# Using validation set to observe the accuracy during model training
y_val_pred = lgbm_model.predict(X_val)
accuracy = accuracy_score(y_val, y_val_pred)
print("Validation set accuracy after feature selection: ", round(accuracy*100,2), "%\n")

In [ ]:
# np.random.seed(0)
# parameters = {
#         'colsample_bytree': (0.1,0.5,0.8),
#         'max_depth': [5,10,15],
#         'num_leaves': [20,30,40,50,60,70,80],
#         'subsample':(0.5,0.8),
#         'n_estimators':(50,100),
# }

# random = RandomizedSearchCV(lgbm_model, parameters, n_iter=30, cv=5)

# random.fit(X_train,y_train)

# df = pd.concat([pd.DataFrame(random.cv_results_['params']), pd.DataFrame(random.cv_results_['mean_test_score']*100, columns=['Accuracy'])] ,axis=1)
# display(df.sort_values('Accuracy', ascending = False))

In [ ]:

lgbm_model = LGBMClassifier( max_depth=15,
                             num_leaves= 30,
                             n_estimators=100,
                             subsample=0.8,
                             colsample_bytree=0.1
                           )

lgbm_model.fit(X_train,y_train)

y_val_pred = lgbm_model.predict(X_val)
y_prob_lgbm = lgbm_model.predict_proba(X_test)[:, 1]  
accuracy = accuracy_score(y_val, y_val_pred)
print("\nValidation set accuracy after hyperparameter tuning: ", round(accuracy*100,2), "%\n")

y_predicted_lgbm = lgbm_model.predict(X_test)
xgb_model
accuracy = lgbm_model.score(X_test,y_test)*100
print("LGBM Classifier Model Test set accuracy:", round(accuracy,2), "%\n")

conf_matrix = confusion_matrix(y_test, y_predicted_lgbm)
print("LGBM Classifier Model Confusion Matrix: \n")
conf_matrix_plot(conf_matrix)

class_report = pd.DataFrame(classification_report(y_test, y_predicted_lgbm, output_dict=True)).transpose()
print("LGBM Classifier Model Classification Report: \n")
class_report_plot(class_report)

end_time = time.time()
run_time = end_time - start_time

final_memory = process.memory_info().rss
memory_usage = final_memory - initial_memory

print("Memory usage:", memory_usage, 'bytes\n')

print("Runtime:", round(run_time,2), 's')

In [ ]:
fpr1, tpr1, thresh1 = roc_curve(y_test, y_prob_dt, pos_label=1)
fpr2, tpr2, thresh2 = roc_curve(y_test, y_prob_knn, pos_label=1)
fpr3, tpr3, thresh3 = roc_curve(y_test, y_prob_lr, pos_label=1)
fpr4, tpr4, thresh4 = roc_curve(y_test, y_prob_svm, pos_label=1)
fpr5, tpr5, thresh5 = roc_curve(y_test, y_prob_rf, pos_label=1)
fpr6, tpr6, thresh6 = roc_curve(y_test, y_prob_xgboost, pos_label=1)
fpr7, tpr7, thresh7 = roc_curve(y_test, y_prob_lgbm, pos_label=1)

auc1 = round(roc_auc_score(y_test, y_prob_dt), 4)
auc2 = round(roc_auc_score(y_test, y_prob_knn), 4)
auc3 = round(roc_auc_score(y_test, y_prob_lr), 4)
auc4 = round(roc_auc_score(y_test, y_prob_svm), 4)
auc5 = round(roc_auc_score(y_test, y_prob_rf), 4)
auc6 = round(roc_auc_score(y_test, y_prob_xgboost), 4)
auc7 = round(roc_auc_score(y_test, y_prob_lgbm), 4)

print(auc5)

plt.figure(figsize=(10,8))

plt.plot(fpr1,tpr1,color='red', linestyle='-', marker='o', linewidth=3, label= "Decision Tree AUC="+str(auc1))
plt.plot(fpr2,tpr2,color='orange', linestyle='-', marker='o', linewidth=3, label= "K Nearest Neighbor AUC="+str(auc2))
plt.plot(fpr3,tpr3,color='yellow', linestyle='-', marker='o', linewidth=3, label= "Logistics Regression AUC="+str(auc3))
plt.plot(fpr4,tpr4,color='green', linestyle='-', marker='o', linewidth=3, label= "Support Vector Machine AUC="+str(auc4))
plt.plot(fpr5,tpr5,color='blue', linestyle='-', marker='o', linewidth=3, label= "Random Forest AUC="+str(auc5))
plt.plot(fpr6,tpr6,color='purple', linestyle='-', marker='o', linewidth=3, label= "XGBoost AUC="+str(auc6))
plt.plot(fpr7,tpr7,color='pink', linestyle='-', marker='o', linewidth=3, label= "Lignt-GBM AUC="+str(auc7))

plt.grid()
plt.legend(loc='best')
plt.title('AUC ROC Plot',size=20)
plt.xlabel('False Positive',size=15)
plt.ylabel('True Positive',size=15)
plt.show()
    


In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import f1_score
from sklearn.metrics import auc

dt_precision, dt_recall, _ = precision_recall_curve(y_test, y_prob_dt)
knn_precision, knn_recall, _ = precision_recall_curve(y_test, y_prob_knn)
lr_precision, lr_recall, _ = precision_recall_curve(y_test, y_prob_lr)
svm_precision, svm_recall, _ = precision_recall_curve(y_test, y_prob_svm)
rf_precision, rf_recall, _ = precision_recall_curve(y_test, y_prob_rf)
xgboost_precision, xgboost_recall, _ = precision_recall_curve(y_test, y_prob_xgboost)
lgbm_precision, lgbm_recall, _ = precision_recall_curve(y_test, y_prob_lgbm)

dt_auc = auc(dt_recall, dt_precision).round(3)
knn_auc = auc(knn_recall, knn_precision).round(3)
lr_auc = auc(lr_recall, lr_precision).round(3)
svm_auc = auc(svm_recall, svm_precision).round(3)
rf_auc = auc(rf_recall, rf_precision).round(3)
xgboost_auc = auc(xgboost_recall, xgboost_precision).round(3)
lgbm_auc = auc(lgbm_recall, lgbm_precision).round(3)

plt.figure(figsize=(10,8))
plt.plot(dt_recall,dt_precision,color='red', linestyle='--', marker='o', linewidth=3, label= "Decision Tree AUC="+str(dt_auc))
plt.plot(knn_recall,knn_precision,color='orange', linestyle='--', marker='o', linewidth=3, label= "K Nearest Neighbor AUC="+str(knn_auc))
plt.plot(lr_recall,lr_precision,color='yellow', linestyle='--', marker='o', linewidth=1, label= "Logistics Regression AUC="+str(lr_auc))
plt.plot(svm_recall,svm_precision,color='green', linestyle='--', marker='o', linewidth=3, label= "Support Vector Machine AUC="+str(svm_auc))
plt.plot(rf_recall,rf_precision,color='blue', linestyle='--', marker='o', linewidth=3, label= "Random Forest AUC="+str(rf_auc))
plt.plot(xgboost_recall,xgboost_precision,color='purple', linestyle='--', marker='o', linewidth=3, label= "XGBoost AUC="+str(xgboost_auc))
plt.plot(lgbm_recall,lgbm_precision,color='pink', linestyle='--', marker='o', linewidth=3, label= "Lignt-GBM AUC="+str(lgbm_auc))


plt.grid()
plt.title('PR Curve',size=20)
plt.legend(loc='best')
plt.xlabel('Recall',size=15)
plt.ylabel('Precision',size=15)
plt.show()



## Deploy the highest accuracy model ( LGBM Classfier Model)

In [ ]:
import pickle

pickle.dump(knn_model, open('model.pkl', 'wb'))

model = pickle.load(open('model.pkl','rb'))